In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import inspect
import getpass
import json

import pandas as pd

project_path = os.path.abspath( os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) + "/../" )
sys.path.insert(0, project_path)

from database_storage.helper import *

from database_storage.database.database import *
from database_storage.main import *

from database_storage.objects.human import *
from database_storage.objects.issue import *
from database_storage.objects.kpi import *
from database_storage.objects.machine import *
from database_storage.objects.tag import *
from database_storage.objects.maintenanceworkorder import *


/Users/sam11/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/sam11/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Setup Dataset informations

foreach Dataset get all the specific information such as:

    bolt : the url of the database server {e.g.: bolt://localhost:7687}
    user : Identification to connect to the database -STRING
    password : Password to connect to the database -STRING
    
    nestor_file: the h5 files generated by Nestor (the tagging app)
    csvSchemas_file : Path of the file use to link the header of the CSV with the properties in the database -YAML
    
------------------------------------
    
    vocab1g_file : Path of the vocab 1G file created by Nestor -CSV
    vocabNg_file : Path of the vocab NG file created by Nestor -JSON
    
    tree_path : Path of the item-item hierarchy (created by <insert a name here>) -JSON

In [3]:
bolt = input('bolt:')
user = input('user:')
password = getpass.getpass("Password:")

nestor_file = input('path of the nestor file:')
csvSchemas_file = input('path of the csv header links:')

bolt:l
user:l
Password:········
path of the nestor file:
path of the csv header links:


# Load Files from Path

foreach path file, load them into the memory using the needed format:
    
    Dataframe for : csv_file, bincsv_file, binNcsv_file, vocabcsv_file, vocabNcsv_file
   
    Dictionary for : csvSchemas_file, tree_path

In [4]:
dataframe = pd.read_hdf(nestor_file, key="df")
dataframe.fillna("", inplace=True)
csvSchemas_dict = openYAMLFile(csvSchemas_file)


dataframe_tag = pd.read_hdf(nestor_file, key="tags")
dataframe_rel = pd.read_hdf(nestor_file, key="rels")


dataframe_vocab1g = None
dataframe_vocabNg = None

/Users/sam11/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/sam11/anaconda3/lib/python3.6/site-packages/cycler-0.10.0-py3.6.egg'

In [ ]:
with open(tree_path) as file:
    tree_dict = json.load(file)

# ConnectGraphDatabase

In [21]:
databaseSchema_file = os.path.join(project_path ,'database_storage','database', 'DatabaseSchema.yaml')
databaseSchema_dict = openYAMLFile(databaseSchema_file)
database = DatabaseNeo4J(bolt, user, password) 

yaml file open
['/Users/sam11/Git/nestor', '', '/Users/sam11/anaconda3/lib/python36.zip', '/Users/sam11/anaconda3/lib/python3.6', '/Users/sam11/anaconda3/lib/python3.6/lib-dynload', '/Users/sam11/anaconda3/lib/python3.6/site-packages', '/Users/sam11/anaconda3/lib/python3.6/site-packages/aeosa', '/Users/sam11/anaconda3/lib/python3.6/site-packages/IPython/extensions', '/Users/sam11/.ipython']


TypeError: expected str, bytes or os.PathLike object, not NoneType

## Clean the database

In [ ]:
database.deleteData()

database.dropConstraints()
database.dropIndexes()

database.createIndexes()
database.createConstraints()

## Create Cypher Queries

### Create historical_data

In [ ]:
database.runQueries(cypherCreate_historicalMaintenanceWorkOrder (database.schema, dataframe, csvSchemas_dict))

### Create tag_data

In [ ]:
database.runQueries(cypherCreate_tag(database.schema, dataframe_tag, dataframe_vocab1g, dataframe_vocabNg))

In [ ]:
database.runQueries(cypherCreate_tag(database.schema, dataframe_rel, dataframe_vocab1g, dataframe_vocabNg))

### Update 1gram-->Ngram

In [ ]:
database.runQueries(cypherLink_Ngram1gram(database.schema))

### Update issue-->item

In [ ]:
database.runQueries(cypherLink_itemIssue(database.schema))

### Update item-->item

In [ ]:
database.runQueries(cypherCreate_itemsTree(database.schema,tree_dict ))

------------------